In [3]:
import numpy as np
import pandas as pd
from math import cos, sin, sqrt
from rear_frontal_intersection import AandL, spatialRearIntersection, rearAndFrontal

f = 150
m = 10000
path = 'input4.xlsx'
df = pd.read_excel(path, engine='openpyxl', dtype=float)
coordination = df.to_numpy()
GCP_left = coordination[0:4,0:2] 
GCP_right = coordination[0:4,2:4]
GCP_ground = coordination[0:4,4:7]
unknown_left = coordination[4:9, 0:2]
unknown_right = coordination[4:9, 2:4]

In [5]:
EO_left = spatialRearIntersection(m, f, GCP_left, GCP_ground)
EO_right = spatialRearIntersection(m, f, GCP_right, GCP_ground)
t = np.hstack((EO_left, EO_right))
X = rearAndFrontal(m=m, f=f,
        GCP_left=GCP_left, GCP_right=GCP_right, GCP_ground=GCP_ground,
        unknown_left=unknown_left, unknown_right=unknown_right
    )

单位权中误差：
 0.006577921632405701
单位权中误差：
 0.007798151090396662
单位权中误差：
 0.006577921632405701
单位权中误差：
 0.007798151090396662


In [19]:
temp_left = AandL(f, t[0:6], 
            image = np.vstack((GCP_left, unknown_left)), 
            ground = np.vstack((GCP_ground, X))
        )
temp_right = AandL(f, t[6:12], 
            image = np.vstack((GCP_right, unknown_right)), 
            ground = np.vstack((GCP_ground, X))
        )

A_1 = temp_left[:,0:6]
A_2 = temp_right[:,0:6]
L = np.hstack((temp_left[:,6], temp_right[:,6]))

A = np.hstack((
            np.vstack((A_1, np.zeros_like(A_1))), 
            np.vstack((np.zeros_like(A_1), A_2)), 
        ))

In [24]:
A_1 @ EO_left

array([-560.11469817, -670.04237416, -564.19381043, -644.09751538,
       -613.59620552, -558.40632158, -609.95430583, -664.2848837 ,
       -606.45962167, -563.00736731, -587.57012512, -360.70538339,
       -364.49825975, -575.11456363, -456.35822516, -465.30316452,
       -474.5049508 , -363.44137264])

In [18]:

n = GCP_left.shape[0] + unknown_left.shape[0]
B = np.zeros((4*n, 3*n))
for i in range(n):
    B[i, i*3:i*3+3] = -temp_left[i, 0:3]
    B[i+n, i*3:i*3+3] = -temp_left[i+n, 0:3]
    B[i+2*n, i*3:i*3+3] = -temp_right[i, 0:3]
    B[i+3*n, i*3:i*3+3] = -temp_right[i+n, 0:3]

np.hstack((A, B))@np.hstack((t, GCP_ground.ravel(), X.ravel())) 

L


array([ 1.71248595e-03,  1.05979791e-03, -5.29904273e-03,  2.47227976e-03,
        3.23513689e-06, -3.97679416e-04, -3.17412117e-04,  2.75162205e-04,
       -2.31217256e-04, -3.03669407e-03,  1.41347767e-03, -3.41228990e-03,
        5.04253439e-03,  3.72623372e-05, -4.29061638e-03, -3.69894870e-03,
        3.49438730e-03, -2.72460929e-03, -3.54697167e-03,  1.26351119e-03,
       -3.96974816e-03,  6.29650423e-03, -4.55045819e-06,  5.69288221e-04,
        4.50919893e-04, -3.87838582e-04,  3.31709411e-04, -9.87674536e-04,
       -2.22562088e-03,  6.13105784e-03, -2.91911397e-03, -3.60223409e-05,
        4.23579260e-03,  3.63889004e-03, -3.42531791e-03,  2.72673475e-03])